# 第三週建議作業

1. 延續關於「機率與疾病」的討論
  1. 假定Pr(C)=0.1，Pr(fever)=0.2，一個發燒病人得到肺炎的機率Pr(C|fever)是多少？
    1. 計算Pr(C|fever)/Pr(C)。這一個比例增加多少倍？
  1. 假定Pr(fever|¬C)=0.01、Pr(C)=0.1，一個發燒病人得到肺炎的機率Pr(C|fever)是多少？
    1. Pr(C|fever) 還是 Pr(¬C|fever) 比較高？
1. 比較 BFS 和 IDS
  1. 參考 AIMA Fig. 3.12 和相關說明
    1. 假定一個搜尋問題，每一個節點有兩個子節點
    1. 利用BFS來搜尋答案的話，假定問題的答案在search tree 的第三層(層數從零開始)的話，最多需要進入 Fig. 3.7 graph search 演算法中的 loop do，做幾次的 choose ？
  1. 參考 AIMA Fig. 3.18 和 Fig. 3.19 和相關說明和 p. 90 上的計算
    1. 假定一個搜尋問題，每一個節點有兩個子節點
    1. 利用IDS來搜尋答案的話，假定問題的答案在search tree 的第三層(層數從零開始)的話，最多需要 choose 多少節點才會找到答案？ 
1. 參照 ai.ch3.dfs.bfs.pdf 第29頁投影片上的 8 puzzle 問題
  1. 利用AIMA書本的 h1 和 h2 來找這一 8 puzzle 問題的答案
  1. 思考一下，如果使用 BFS 或者 DFS 的話，所需要 expand 的節點的數目會不會多很多？